## Predictive Fraud Detection 

In [ ]:
import pandas as pd
import numpy as np
import DR_DB_Cred as cred

In [2]:
import cx_Oracle

In [3]:
# Define Oracle Engine
# engine = create_engine("mysql+pymysql://" + useruat + ":" + passuat + "@" + hostuat + "/" + dbuat, pool_size = 20, max_overflow = 0 )
dsn = cx_Oracle.makedsn(cred.hostdr, cred.portdr, service_name=cred.servicedr)
or_conn = cx_Oracle.connect(user=cred.userdr, password=cred.passdr, dsn=dsn)

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html for help

In [4]:
df=pd.read_pickle("data_30.01.2024-24.06.2024.pkl")

In [13]:
main_df=pd.read_pickle("Oct_data.pkl")

In [5]:
df['STATUS'].unique()

array(['claimPackageApprovedandPaid', 'preAuthApproved',
       'claimPackageRejected', 'claimPackageApproved',
       'preAuthPackageRejected', 'claimapprovalpending',
       'claimApprovalPendingAtAnalyser', 'claimRejectedByAnalyser',
       'claimQueryByAnalyser', 'revertBackToAnalyserByClaim'],
      dtype=object)

In [ ]:
df.columns

Index(['TID', 'TPA_NAME', 'TID_CREATION_DATE', 'ADMISSION_NO', 'PACKAGE_CODE',
       'PKG_ID', 'PACKAGE_BOOKED_DATE', 'PACKAGE_RATE', 'BOOKED_AMOUNT',
       'APPROVED_AMOUNT', 'HOSPITAL_CODE', 'HOSPITAL_NAME', 'WALLET_TYPE',
       'PACKAGE_TYPE', 'IDENTITY_STATUS', 'STATUS', 'DATE_OF_ADMISSION',
       'DATE_OF_DISCHARGE', 'RNK'],
      dtype='object')

### Data Processing

In [10]:
df['DATE_OF_ADMISSION'].isna().sum()

0

In [11]:
df.isnull().sum()

TID                        0
TPA_NAME                   0
TID_CREATION_DATE          0
ADMISSION_NO               0
PACKAGE_CODE               0
PKG_ID                     0
PACKAGE_BOOKED_DATE        0
PACKAGE_RATE               0
BOOKED_AMOUNT              0
APPROVED_AMOUNT            0
HOSPITAL_CODE              0
HOSPITAL_NAME              0
WALLET_TYPE                0
PACKAGE_TYPE               0
IDENTITY_STATUS            0
STATUS                     0
DATE_OF_ADMISSION          0
DATE_OF_DISCHARGE      59293
RNK                        0
dtype: int64

In [25]:
# df.groupby('ADMISSION_NO')
df[~df['TID'].duplicated()].head(2)#isin(df['TID'].unique())].head(2)

,TID,TPA_NAME,TID_CREATION_DATE,ADMISSION_NO,PACKAGE_CODE,PKG_ID,PACKAGE_BOOKED_DATE,PACKAGE_RATE,BOOKED_AMOUNT,APPROVED_AMOUNT,HOSPITAL_CODE,HOSPITAL_NAME,WALLET_TYPE,PACKAGE_TYPE,IDENTITY_STATUS,STATUS,DATE_OF_ADMISSION,DATE_OF_DISCHARGE,RNK
0,T01022428122247,VIDAL,2024-01-31,69823101243041810,2831-CA00001207000049,00,2024-02-01,40000,0.0,40000.0,Jaip248,Sawai Man Singh Hospital,NIA,BP,NFSA,claimPackageApprovedandPaid,2024-01-31 16:51:00,2024-02-02 17:17:00,1
3,T01022428122305,VIDAL,2024-01-30,76623001243033198,1834-MG0000250000274-STR1109,00,2024-02-01,1800,0.0,1800.0,Jaip157,NIMS MEDICAL COLLEGE HOSPITAL,NIA,IP,NFSA,claimPackageApprovedandPaid,2024-01-30 18:06:00,2024-02-02 17:39:00,1


In [5]:
# main_df[main_df['PACKAGE_RATE'].isna()]

In [6]:
# main_df[main_df['PACKAGE_CODE'].isna()]

In [5]:
df1=df.copy()

In [6]:
df1['TID_CREATION_DATE'] = pd.to_datetime(df1['TID_CREATION_DATE'])

### --- Patient-Level Features ---

In [7]:
# Claims count per patient
patient_claim_counts = df1.groupby('ADMISSION_NO')['TID'].count().rename('claims_count').reset_index()
df1 = df1.merge(patient_claim_counts, on='ADMISSION_NO', how='left')

In [8]:
# Average time between claims per patient
# df1.groupby('ADMISSION_NO')['PACKAGE_BOOKED_DATE'].shift(1)
claim_level_data = df1.groupby(['TID','PACKAGE_CODE', 'PKG_ID']).agg({
    'BOOKED_AMOUNT': 'sum',                       # Total booked amount per claim
    'APPROVED_AMOUNT': 'sum',                     # Total approved amount per claim
    'PACKAGE_RATE': 'mean',                       # Average package rate per claim
    'STATUS': 'nunique',                          # Count of unique statuses per claim
    'DATE_OF_ADMISSION': 'min',                   # Earliest admission date
    'DATE_OF_DISCHARGE': 'max',                   # Latest discharge date
    'TID_CREATION_DATE': 'min',                   # Earliest TID creation date
}).reset_index()

In [43]:
# df1.groupby()
claim_level_data['TID'].nunique()

1233713

### --- Additional Calculations on Aggregated Claim Data ---

In [9]:
# Admission duration (DATE_OF_DISCHARGE - DATE_OF_ADMISSION)
claim_level_data['admission_duration'] = (claim_level_data['DATE_OF_DISCHARGE'] - claim_level_data['DATE_OF_ADMISSION']).dt.days

In [10]:
# Time to Package Booking (DATE_OF_ADMISSION - PACKAGE_BOOKED_DATE)
claim_level_data['admission_to_booking_lag'] = (claim_level_data['DATE_OF_ADMISSION'] - df1['PACKAGE_BOOKED_DATE']).dt.days


In [11]:
# Claim Submission Lag (TID_CREATION_DATE - DATE_OF_ADMISSION)
claim_level_data['claim_submission_lag'] = (claim_level_data['TID_CREATION_DATE'] - claim_level_data['DATE_OF_ADMISSION']).dt.days

In [11]:
claim_level_data['APPROVED_AMOUNT']

0          8500.0
1             0.0
2          1800.0
3             0.0
4             0.0
            ...  
2650208       0.0
2650209    1500.0
2650210     600.0
2650211    1500.0
2650212       0.0
Name: APPROVED_AMOUNT, Length: 2650213, dtype: float64

In [14]:
# Deviation of Approved amount from Package Rate 
claim_level_data['rate_deviation'] = claim_level_data['APPROVED_AMOUNT'] - claim_level_data['PACKAGE_RATE']

In [16]:
# Approved-to-Booked Ratio
# wherever is the Booked amount zero Package rate has been take in account
claim_level_data['BOOKED_AMOUNT'].fillna(0).astype(int).apply(lambda x : claim_level_data['PACKAGE_RATE'] if x == 0 else claim_level_data['BOOKED_AMOUNT'])
# claim_level_data['approved_to_booked_ratio'] = claim_level_data['APPROVED_AMOUNT'] / claim_level_data['BOOKED_AMOUNT']


MemoryError: Unable to allocate 20.2 MiB for an array with shape (2650213,) and data type float64

In [14]:
# claim_level_data['PACKAGE_RATE'].tolist()

In [20]:
for x,y in enumerate(claim_level_data['BOOKED_AMOUNT'].tolist()):
    # print(x,y)
    if y==0:
        claim_level_data['BOOKED_AMOUNT']=claim_level_data['PACKAGE_RATE']

KeyboardInterrupt: 

In [ ]:
clai